In [2]:
import requests
import pandas as pd
import os
from datetime import datetime
import re

# Download data

## Image data

In [16]:
url_images = 'http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers = {'AccountKey' : os.environ["LTA_ACCOUNT_KEY"]}
images = requests.get(url_images, headers = headers).json()

{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#Traffic-Imagesv2',
 'value': [{'CameraID': '1001',
   'Latitude': 1.29531332,
   'Longitude': 103.871146,
   'ImageLink': 'https://dm-traffic-camera-itsc.s3.ap-southeast-1.amazonaws.com/2022-11-16/22-25/1001_2223_20221116222501_984955.jpg?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEA0aDmFwLXNvdXRoZWFzdC0xIkcwRQIhAJfZgTzkrUh8sfyc0YJ9lnozKhzR8rO3%2FEXipOu7VDz5AiABu7zl2c0QTuNdwXi9pJ1wuwywSOhyQIlqFAG%2Bgq1lWCrbBAgWEAMaDDM0MDY0NTM4MTMwNCIMhvTfxaKsrG43hmFXKrgEx9QbbhaRHEdkmKQKdH%2FviPh7uXMXVmctEJZ8pFrWTvdh6jTCkZbR1%2B5OTFhNKIVGk9mVJbSOSr4Jcwl%2FWjfpDpnfSvlrGMZvFBMYbJ0MoNOxZdwrzwyQCBAzpDpfCQRUtlEEKJnUMjxWFQm5lj9wN7vdvpPnmNw0JEcyh1jOqIDFCd%2FGjTQriwF4MH12zpr97WmVMTT6MkSApqJZ6kEA25IOa7I2zdbmaHH%2Fj8NOnCuFsLONXrjQuvcBhGiQImmmEKYCxSHdPerTvwsJ%2Ft6FBzHPK1W8Wq3jFE0iUhWwcpMaHJljkxpcLU9rxqDHxrY2cEUtcPEy3ivNEYD8QhWyZ%2BjyqaoLWDvoUpJgIbct6bP1d5pAIrp4N40tbKRbAE4uL2kheXvd64KcltQOXVVnONnUE7Jc94zOaJWuJXKFpE8sXNnU%2Fa5q04Z86vrH1Bu2d8pIDH

In [19]:
filename_regex = re.compile(r'/([\d_]+.jpg)')

for camera in images['value']:
    url = camera['ImageLink']
    # Make filename from imagelink URL which contains camera ID, date and time of image
    filename = os.path.join(os.path.expanduser(os.environ.get("LOCAL_DATA_PATH")), 
                            'data-images', 
                            re.findall(filename_regex, url)[0]) 
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

In [25]:
url_speeds = 'http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2'
speeds = requests.get(url_speeds, headers = headers).json()
speedsdata = [pd.DataFrame(speeds['value']).set_index('LinkID')]
for skip in range(500,59500,500):
    new_speeds = requests.get(url_speeds, headers = headers, params = {'$skip': f'{skip}'}).json()
    speedsdata.append(pd.DataFrame(new_speeds['value']).set_index('LinkID'))
speedsdf = pd.concat(speedsdata)
filename = os.path.join(os.path.expanduser(os.environ.get("LOCAL_DATA_PATH")), 
                        'data-speeds', 
                        datetime.now().strftime('%Y-%m-%d_%H%M') + 'speed.csv')
speedsdf.to_csv(filename)

# Cleaning & Matching

In [ ]:
imagesdf = pd.DataFrame(images['value']).set_index('CameraID')

In [74]:
speedsdf[['lat1', 'long1', 'lat2', 'long2']] = speedsdf['Location'].str.split(expand = True)

In [75]:
speedsdf[['lat1', 'long1', 'lat2', 'long2']] = speedsdf[['lat1', 'long1', 'lat2', 'long2']].apply(pd.to_numeric)

In [72]:
# speedsdf.loc[speedsdf['lat2'] < speedsdf['lat1'], ['lat1', 'lat2']] = speedsdf.loc[speedsdf['lat2'] < speedsdf['lat1'], ['lat2', 'lat1']].values

# Check if speed exists for each camera

In [119]:
imagesdf['speeds'] = None

TODO: make this more efficient by sorting, if need to do multiple times

In [120]:
%%time
for cameraindex, cameradata in imagesdf.iterrows():
    for speedindex, speeddata in speedsdf.iterrows():
        if (((speeddata['lat1'] <= cameradata['Latitude'] <= speeddata['lat2'])
             or (speeddata['lat1'] >= cameradata['Latitude'] >= speeddata['lat2']))
            and ((speeddata['long1'] <= cameradata['Longitude'] <= speeddata['long2'])
                 or (speeddata['long1'] >= cameradata['Longitude'] >= speeddata['long2']))):
            print('found')
            if imagesdf.at[cameraindex,'speeds']:
                imagesdf.at[cameraindex, 'speeds'].append(speedindex)
            else:
                imagesdf.at[cameraindex, 'speeds'] = [speedindex]
            print(imagesdf.at[cameraindex, 'speeds'])

found
['103100098']
found
['103100098', '103100099']
found
['107002716']
found
['107002716', '107002900']
found
['107002880']
found
['108002954']
found
['108002954', '108002955']
found
['113000212']
found
['113000292']
found
['113000192']
found
['113000260']
found
['103055618']
found
['103055618', '103055619']
found
['103055618', '103055619', '112000000']
found
['104005996']
found
['116000948']
found
['116000948', '116000976']
found
['103104150']
found
['103104150', '103104151']
found
['103115296']
found
['103013056']
found
['103013056', '103061773']
found
['103013056', '103061773', '103103406']
found
['103013056', '103061773', '103103406', '103103407']
found
['103013056', '103061773', '103103406', '103103407', '116000006']
found
['103013056', '103061773', '103103406', '103103407', '116000006', '116000007']
found
['103066482']
found
['103066482', '103066483']
found
['103066482', '103066483', '103116220']
found
['103066670']
found
['103066670', '103066671']
found
['103066670', '10306667

In [134]:
imagesdf[imagesdf['speeds'].notnull()]

,Latitude,Longitude,ImageLink,speeds
CameraID,,,,
1001,1.295313,103.871146,https://dm-traffic-camera-itsc.s3.ap-southeast...,"[103100098, 103100099]"
1003,1.323957,103.872858,https://dm-traffic-camera-itsc.s3.ap-southeast...,"[107002716, 107002900]"
1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,[107002880]
1006,1.357099,103.902042,https://dm-traffic-camera-itsc.s3.ap-southeast...,"[108002954, 108002955]"
1501,1.274144,103.851317,https://dm-traffic-camera-itsc.s3.ap-southeast...,[113000212]
...,...,...,...,...
9702,1.394741,103.817971,https://dm-traffic-camera-itsc.s3.ap-southeast...,"[103104226, 103104227]"
9703,1.422857,103.773005,https://dm-traffic-camera-itsc.s3.ap-southeast...,"[103055720, 106003820]"
9704,1.422143,103.795421,https://dm-traffic-camera-itsc.s3.ap-southeast...,"[104006714, 104006715, 106003612]"


In [132]:
camera_speed_map = {}
for index, camera in imagesdf.iterrows():
    camera_speed_map[index] = camera['speeds']

In [133]:
camera_speed_map

{'1001': ['103100098', '103100099'],
 '1002': None,
 '1003': ['107002716', '107002900'],
 '1004': ['107002880'],
 '1005': None,
 '1006': ['108002954', '108002955'],
 '1501': ['113000212'],
 '1502': ['113000292'],
 '1503': None,
 '1504': ['113000192'],
 '1505': ['113000260'],
 '1701': None,
 '1702': ['103055618', '103055619', '112000000'],
 '1703': ['104005996'],
 '1704': ['116000948', '116000976'],
 '1705': ['103104150', '103104151'],
 '1706': ['103115296'],
 '1707': ['103013056',
  '103061773',
  '103103406',
  '103103407',
  '116000006',
  '116000007'],
 '1709': None,
 '1711': None,
 '2701': None,
 '2702': None,
 '2703': ['103066482', '103066483', '103116220'],
 '2704': ['103066670', '103066671', '103104628'],
 '2705': None,
 '2706': None,
 '2707': None,
 '2708': ['103104640'],
 '3702': ['103065438', '103065439', '103110492'],
 '3704': None,
 '3705': ['103064922', '103064923', '103095272'],
 '3793': ['103022252', '103086770', '103086771', '103103418', '103103419'],
 '3795': None,
 '3

In [10]:
url_images = 'http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers = {'AccountKey' : 'Ac8SpA/uQO22Wv5fA+lmaQ=='}
images = requests.get(url_images, headers = headers).json()
imagesdf2 = pd.DataFrame(images['value']).set_index('CameraID')

,Latitude,Longitude,ImageLink
CameraID,,,
1001,1.295313,103.871146,https://dm-traffic-camera-itsc.s3.ap-southeast...
1002,1.319541,103.878563,https://dm-traffic-camera-itsc.s3.ap-southeast...
1003,1.323957,103.872858,https://dm-traffic-camera-itsc.s3.ap-southeast...
1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...
1005,1.363520,103.905394,https://dm-traffic-camera-itsc.s3.ap-southeast...
...,...,...,...
9702,1.394741,103.817971,https://dm-traffic-camera-itsc.s3.ap-southeast...
9703,1.422857,103.773005,https://dm-traffic-camera-itsc.s3.ap-southeast...
9704,1.422143,103.795421,https://dm-traffic-camera-itsc.s3.ap-southeast...


In [5]:
imagesdf2.loc['1707',:]

Latitude                                              1.280366
Longitude                                           103.830451
ImageLink    https://dm-traffic-camera-itsc.s3.ap-southeast...
Name: 1707, dtype: object

In [11]:
imagesdf2.loc['1707','ImageLink']

'https://dm-traffic-camera-itsc.s3.ap-southeast-1.amazonaws.com/2022-11-15/12-45/1707_1244_20221115124542_559853.jpg?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEOz%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkgwRgIhANTrNtG%2FLJhHYpWVwr8SRQbN%2FtitOAqjWt1Q4lGu7I93AiEAowxMA6uiTfgjwtO%2FeZLaeg%2FInkcFV1%2FiyZBkKoKWjH8q5AQI5f%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARADGgwzNDA2NDUzODEzMDQiDE3c3Z1ERY5ABNTd2iq4BNIixuwXfU%2BI1CG3O%2FrGpDU2k2FooCMgfAK%2FSHXYL6D6hl4CDuZY6kO%2BQr1trPQSyoNu%2FMO8wJT3%2BTKakne2pyM9KRRjJo61nSLxL2TgnwwUG8JXPFN5krA5TIKTtJBYajXmwdG4UHkYckb5W3%2B3%2F9qZx3Y9Z65WD20L%2FNPCI%2B1jd%2BvDxh2RNv0b%2FTAtNMhPrBXPP0UNIFD%2FOQqJFXQXEK8WWeI9i%2BqArbEdCgt56h%2FOrVcvKB8MEXOcQ4FGNo9x32lbL%2Fb50eFCGHKtSRjvQ3%2FhTtTl8N8HeuPsxfbvVsg%2BboHE6uC3wNxGwNXZjqjNBpo2pgvjMQVqt5IMifUBO2w99reVICZNaH%2FRIEIhMeTh%2FsUgJSeUKQ7GmUQzpkgx3kGoCs109tV6XWX1nLwbzNfTTS3qkBa%2BWx0pK%2BvVlEMUFABH%2FLCYoKiZN3YxeVKhTX8RGuix5FQkT61BKqrNKXDrIM9kyWb6AGLTtnsJLOkQDMQdpeepSxP7SjfFRjXfY2oMEyp7kIqMS5ef75CXKiqD0imGsk2ZkmsklDBeFf

In [9]:
speedsdf.loc[['103013056',
  '103061773',
  '103103406',
  '103103407',
  '116000006',
  '116000007'], :]

,RoadName,RoadCategory,SpeedBand,MinimumSpeed,MaximumSpeed,Location
LinkID,,,,,,
103013056,JALAN BUKIT MERAH,F,5,40,49,1.2797544818379811 103.82916817881473 1.280601...
103061773,CENTRAL EXPRESSWAY,F,5,40,49,1.2793302810560436 103.82958020398998 1.280601...
103103406,CENTRAL EXPRESSWAY,A,8,70,999,1.2806016384288461 103.83045155058139 1.279879...
103103407,CENTRAL EXPRESSWAY,A,8,70,999,1.2798791493486743 103.8296926102903 1.2806016...
116000006,MACALISTER ROAD,E,8,70,999,1.2809400798468304 103.8314402119217 1.2798566...
116000007,MACALISTER ROAD,E,2,10,19,1.279856622402995 103.83029145890723 1.2809400...


# Integrating weather